In [14]:
# import dependencies
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf
from sklearn.decomposition import PCA

In [15]:
merged_nb_df = pd.read_csv("merged_nb_df.csv")
pd.set_option('display.max_columns', None)
merged_nb_df.head()

,size_l,og,fg,abv,ibu,color,boilsize,boiltime,boilgravity,efficiency,mashthickness,pitchrate,primarytemp,review_taste,beer_abv,sugarscale_Plato,sugarscale_Specific Gravity,brewmethod_All Grain,brewmethod_BIAB,brewmethod_Partial Mash,brewmethod_extract
0,18.93,1.082,1.013,9.10,0.00,4.10,21.58,60.0,0.000,72.0,0.0,0.00,0.00,4.103825,7.6,0.0,1.0,1.0,0.0,0.0,0.0
1,7.57,1.076,1.014,8.10,33.51,3.68,8.71,60.0,1.066,75.0,0.0,0.00,0.00,4.103825,7.6,0.0,1.0,1.0,0.0,0.0,0.0
2,20.82,1.043,1.011,4.14,13.22,17.67,28.39,90.0,0.000,85.0,0.0,1.75,12.22,3.028652,4.4,0.0,1.0,1.0,0.0,0.0,0.0
3,22.71,1.069,1.017,6.75,58.76,40.00,29.34,90.0,1.053,70.0,0.0,0.00,0.00,4.265971,6.4,0.0,1.0,1.0,0.0,0.0,0.0
4,22.71,1.068,1.018,6.53,73.40,40.00,27.44,90.0,1.056,65.0,1.3,1.25,18.89,4.265971,6.4,0.0,1.0,1.0,0.0,0.0,0.0


In [16]:
# Since beers with a rating of four or above is the top 34% of beers, let's go with those as "good" beers.
# We need a new category "good beer" with a 1 for good and 0 for bad
merged_nb_df["good_beer"]=0

In [17]:
# for all the "good beers", put a 1 in the good_beer column
merged_nb_df.loc[merged_nb_df["review_taste"].abs()>=4, "good_beer"] = 1

In [18]:
beer_class = merged_nb_df.drop(columns = ["review_taste"])

In [19]:
# Split our preprocessed data into our features and target arrays
y = beer_class.good_beer.values
X = beer_class.drop(columns = ["good_beer"]).values


In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaled = scaler.fit_transform(X)

In [26]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=7)
beer_pca = pca.fit_transform(X_scaled)

In [27]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(data = beer_pca, columns = ["PC 1", "PC 2", "PC 3", "PC4", "PC5", "PC6", "PC7"], index = merged_nb_df.index)
pcs_df.head(25)

,PC 1,PC 2,PC 3,PC4,PC5,PC6,PC7
0,-0.410503,0.626444,-0.623953,1.830314,-0.044380,-1.999782,0.365340
1,-0.235861,0.562563,-0.684679,1.924613,-0.028254,-1.584102,0.306373
2,-0.367957,1.358399,1.120886,-0.970447,-1.237861,1.937976,-1.003943
3,-0.018194,0.466143,-0.493764,3.832140,0.050382,-1.476556,0.040483
4,-0.108942,0.925018,0.211235,3.716335,0.680681,2.116879,-0.261921
5,-0.337504,-1.491203,0.674537,5.872409,-1.398082,-1.478913,-0.020700
6,-0.036047,1.522064,0.492261,-1.365684,-0.778678,1.374065,-0.092217
7,0.052287,1.865721,0.577003,-0.997003,-0.764553,1.403438,-0.789802
8,-0.287691,1.186241,0.905265,-0.029112,-0.602708,1.017625,-0.572644
9,-0.513025,-1.114305,1.927248,1.468240,-1.813978,1.308266,-0.724969


In [28]:
# Get the explained variance ratio
pca.explained_variance_ratio_

array([0.21494731, 0.11213441, 0.09672483, 0.08756904, 0.07330603,
       0.06575426, 0.05447745])

In [30]:
# 5 components explain 56% of the variance
# 7 components explain 67% of the variance (roughly 2/3)
# So let's stick with 7